In [2]:
import pandas as pd
import numpy as np
import re
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')


In [3]:
DATA_DIR = Path(".")  # Current directory

# List all CSV files to verify
csv_files = list(DATA_DIR.glob("*.csv"))
print(f"Found {len(csv_files)} CSV files:")
for f in csv_files:
    print(f"  - {f.name}")

Found 11 CSV files:
  - baseline_outcome_dataset.csv
  - BCVA.csv
  - clinical_full_merged.csv
  - clinical_full_merged2.csv
  - CST.csv
  - Data.csv
  - Demographics.csv
  - DRSS.csv
  - fundus_scanned_files.csv
  - PrimeDR Biomarker Clinical Data Images.csv
  - PrimeDR Clinical Data Images.csv


# Demographic CSV

In [4]:
# Read Demographics.csv
demo_df = pd.read_csv("Demographics.csv", header=[0,1])

print("Shape:", demo_df.shape)
print("\nColumn names:")
print(demo_df.columns.tolist())
print("\n" + "="*80)
print("First 5 rows:")
demo_df.head()

Shape: (40, 20)

Column names:
[('Patient Information', 'Patient \nID'), ('Unnamed: 1', 'Treatment Arm'), ('Unnamed: 2', 'Study\n Eye'), ('Unnamed: 3', 'Age'), ('Unnamed: 4', 'Gender'), ('Unnamed: 5', 'Ethnicity'), ('Unnamed: 6', 'Race'), ('Diabetes', 'Type of\n Diabetes'), ('Unnamed: 8', 'Number of Years with Diabetes'), ('Unnamed: 9', 'Baseline HbA1c'), ('Unnamed: 10', 'W24 HbA1c'), ('Unnamed: 11', 'W52 HbA1c'), ('Unnamed: 12', 'W76 HbA1c'), ('Unnamed: 13', 'W104 HbA1c'), ('Unnamed: 14', 'BMI (kg/m^2)'), ('Baseline', 'ETDRS BCVA'), ('Unnamed: 16', 'CST'), ('Unnamed: 17', 'Injection'), ('Unnamed: 18', 'DRSS'), ('Unnamed: 19', 'Leakage Index')]

First 5 rows:


,Patient Information,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Diabetes,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Baseline,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
,Patient \nID,Treatment Arm,Study\n Eye,Age,Gender,Ethnicity,Race,Type of\n Diabetes,Number of Years with Diabetes,Baseline HbA1c,W24 HbA1c,W52 HbA1c,W76 HbA1c,W104 HbA1c,BMI (kg/m^2),ETDRS BCVA,CST,Injection,DRSS,Leakage Index
0,01-001,2,OS,44,M,N H/L,White,2,20,7.1,8.7,8.4,9.1,8.4,NaN,97,275,Yes,53,1.59
1,01-002,2,OD,56,F,N H/L,White,2,25,11.3,9.1,,NaN,NaN,34.484657,68,238,Yes,47,3.56
2,01-013,2,OD,38,M,H/L,White,1,13,8.0,9.5,9.6,11.7,9.8,25.997929,88,303,Yes,53,5.08
3,01-014,2,OS,55,M,N H/L,White,2,12,10.1,7.2,7.4,7.4,7.4,31.871377,95,256,Yes,53,2.34
4,01-023,2,OD,56,M,H/L,White,2,22,7.1,6.7,5.9,6.9,NaN,35.669938,81,267,Yes,61,4.67


In [5]:
# Read with multi-level headers
demo_df = pd.read_csv("Demographics.csv", header=[0,1])

# Flatten the multi-level columns properly
# Strategy: Use level 1 (specific names), but if it's empty, use level 0
new_cols = []
for col in demo_df.columns:
    level0, level1 = col[0], col[1]
    
    # Clean whitespace
    level0 = level0.strip() if isinstance(level0, str) else str(level0)
    level1 = level1.strip() if isinstance(level1, str) else str(level1)
    
    # If level1 has actual content (not just whitespace), use it
    # Otherwise use level0
    if level1 and level1 != 'nan' and not level1.startswith('Unnamed'):
        new_cols.append(level1)
    elif level0 and not level0.startswith('Unnamed'):
        new_cols.append(level0)
    else:
        new_cols.append(f"{level0}_{level1}")

demo_df.columns = new_cols

print("Column names after flattening:")
print(demo_df.columns.tolist())
print("\nFirst 3 rows:")
demo_df.head(3)

Column names after flattening:
['Patient \nID', 'Treatment Arm', 'Study\n Eye', 'Age', 'Gender', 'Ethnicity', 'Race', 'Type of\n Diabetes', 'Number of Years with Diabetes', 'Baseline HbA1c', 'W24 HbA1c', 'W52 HbA1c', 'W76 HbA1c', 'W104 HbA1c', 'BMI (kg/m^2)', 'ETDRS BCVA', 'CST', 'Injection', 'DRSS', 'Leakage Index']

First 3 rows:


,Patient \nID,Treatment Arm,Study\n Eye,Age,Gender,Ethnicity,Race,Type of\n Diabetes,Number of Years with Diabetes,Baseline HbA1c,W24 HbA1c,W52 HbA1c,W76 HbA1c,W104 HbA1c,BMI (kg/m^2),ETDRS BCVA,CST,Injection,DRSS,Leakage Index
0,01-001,2,OS,44,M,N H/L,White,2,20,7.1,8.7,8.4,9.1,8.4,NaN,97,275,Yes,53,1.59
1,01-002,2,OD,56,F,N H/L,White,2,25,11.3,9.1,,NaN,NaN,34.484657,68,238,Yes,47,3.56
2,01-013,2,OD,38,M,H/L,White,1,13,8.0,9.5,9.6,11.7,9.8,25.997929,88,303,Yes,53,5.08


In [6]:
# Check basic info
print("="*80)
print("DEMOGRAPHICS DATA SUMMARY")
print("="*80)

print(f"\nTotal patients: {len(demo_df)}")
print(f"\nUnique Patient IDs: {demo_df.iloc[:, 0].nunique()}")
print(f"\nFirst column values (Patient IDs):")
print(demo_df.iloc[:5, 0].values)

# check cats and dist.
print(f"\n--- Categorical Variables ---")
if 'Study_Eye' in demo_df.columns or 'Study\n Eye' in demo_df.columns:
    eye_col = 'Study_Eye' if 'Study_Eye' in demo_df.columns else demo_df.columns[2]
    print(f"\nStudy Eye distribution:")
    print(demo_df[eye_col].value_counts())

# Show numeric columns
print(f"\n--- Numeric Variables (summary) ---")
numeric_cols = demo_df.select_dtypes(include=[np.number]).columns
print(f"Numeric columns: {list(numeric_cols)}")

DEMOGRAPHICS DATA SUMMARY

Total patients: 40

Unique Patient IDs: 40

First column values (Patient IDs):
['01-001' '01-002' '01-013' '01-014' '01-023']

--- Categorical Variables ---

Study Eye distribution:
Study\n Eye
OD    24
OS    16
Name: count, dtype: int64

--- Numeric Variables (summary) ---
Numeric columns: ['Treatment Arm', 'Age', 'Type of\n Diabetes', 'Number of Years with Diabetes', 'Baseline HbA1c', 'W24 HbA1c', 'W76 HbA1c', 'W104 HbA1c', 'BMI (kg/m^2)', 'ETDRS BCVA', 'CST', 'DRSS', 'Leakage Index']


In [7]:
# Basic statistics on numerics
print("\nBasic statistics:")
print(demo_df.describe())


Basic statistics:
       Treatment Arm        Age  Type of\n Diabetes  \
count       40.00000  40.000000           40.000000   
mean         1.50000  50.825000            1.875000   
std          0.50637  10.876669            0.334932   
min          1.00000  25.000000            1.000000   
25%          1.00000  45.000000            2.000000   
50%          1.50000  51.500000            2.000000   
75%          2.00000  56.500000            2.000000   
max          2.00000  72.000000            2.000000   

       Number of Years with Diabetes  Baseline HbA1c  W24 HbA1c  W76 HbA1c  \
count                      40.000000         40.0000  30.000000  20.000000   
mean                       15.825000          9.1150   8.503333   9.215000   
std                         8.421241          2.2147   1.788562   2.324305   
min                         0.000000          5.1000   4.600000   6.500000   
25%                        11.000000          7.5500   7.400000   7.325000   
50%              

In [8]:
print(f"\n--- Missing Data ---")
print(demo_df.isnull().sum())


--- Missing Data ---
Patient \nID                      0
Treatment Arm                     0
Study\n Eye                       0
Age                               0
Gender                            0
Ethnicity                         0
Race                              0
Type of\n Diabetes                0
Number of Years with Diabetes     0
Baseline HbA1c                    0
W24 HbA1c                        10
W52 HbA1c                        11
W76 HbA1c                        20
W104 HbA1c                       19
BMI (kg/m^2)                      1
ETDRS BCVA                        0
CST                               0
Injection                         0
DRSS                              0
Leakage Index                     0
dtype: int64


Key findings:

✅ 40 eyes (24 OD, 16 OS) - this matches your thesis document

✅ Age range: 25-72 years (mean ~51)

    years with Diabetes: 0-33 yrs

    baseline Hba1c: 5.1 ~ 14.7

✅ Baseline BCVA: 83.3 ± 7.9 letters

✅ Baseline CST: 270 ± 24 μm

⚠️ Some missing HbA1c at follow-up visits (but baseline is complete)

# PrimeDR Clinical Data Images.csv

In [9]:
# Load PrimeDR Clinical Data Images.csv
import pandas as pd
clinical_df = pd.read_csv("PrimeDR Clinical Data Images.csv")

print("="*80)
print("CLINICAL DATA IMAGES (32K+ images with labels)")
print("="*80)

print(f"\nShape: {clinical_df.shape}")
print(f"\nColumns: {clinical_df.columns.tolist()}")


print(f"\n--- Data Summary ---")
print(f"Total image records: {len(clinical_df)}")
print(f"Unique patients: {clinical_df['Patient_ID'].nunique()}")
print(f"Unique eyes: {clinical_df['Eye_ID'].nunique()}")
clinical_df.head()


CLINICAL DATA IMAGES (32K+ images with labels)

Shape: (32337, 5)

Columns: ['File_Path', 'BCVA', 'CST', 'Eye_ID', 'Patient_ID']

--- Data Summary ---
Total image records: 32337
Unique patients: 40
Unique eyes: 40


,File_Path,BCVA,CST,Eye_ID,Patient_ID
0,/Prime_FULL/01-001/W0/OS/27.png,97.0,275.0,58,58
1,/Prime_FULL/01-001/W0/OS/0.png,97.0,275.0,58,58
2,/Prime_FULL/01-001/W0/OS/1.png,97.0,275.0,58,58
3,/Prime_FULL/01-001/W0/OS/10.png,97.0,275.0,58,58
4,/Prime_FULL/01-001/W0/OS/11.png,97.0,275.0,58,58


we have 40 patients, 40 eyes (not 80) = 24 OD and 16 OS

In [10]:
# Extract patientID, week, eye from file path
import re

# 1. Extract Patient ID (string format like "01-001")
clinical_df['Patient_ID_String'] = clinical_df['File_Path'].str.extract(r'/(\d{2}-\d{3})/')

# 2. Extract Week number (already done, but let's redo for completeness)
def extract_week_from_path(path):
    """Extract week number from path like '/Prime_FULL/01-001/W0/OS/27.png'"""
    if pd.isna(path):
        return None
    match = re.search(r'/W(\d+)/', str(path))
    return int(match.group(1)) if match else None

clinical_df['Week'] = clinical_df['File_Path'].apply(extract_week_from_path)

# 3. Extract Eye (OD or OS)
clinical_df['Eye'] = clinical_df['File_Path'].str.extract(r'/(OD|OS)/')

# 4. Extract B-scan number (or filename)
clinical_df['Filename'] = clinical_df['File_Path'].str.split('/').str[-1]
clinical_df['B_scan_number'] = clinical_df['Filename'].str.extract(r'(\d+)\.')[0]
clinical_df['B_scan_number'] = pd.to_numeric(clinical_df['B_scan_number'], errors='coerce')

# 5. Extract file extension (to distinguish OCT vs Fundus)
clinical_df['Image_Type'] = clinical_df['File_Path'].str.extract(r'\.(\w+)$')[0]

# Verify extraction
print("\n✓ Extraction complete!")
print(f"\nNew columns added: Patient_ID_String, Week, Eye, Filename, B_scan_number, Image_Type")

cols_to_check = ['File_Path', 'Patient_ID_String', 'Week', 'Eye', 
                 'B_scan_number', 'Image_Type', 'Eye_ID', 'Patient_ID']
clinical_df




✓ Extraction complete!

New columns added: Patient_ID_String, Week, Eye, Filename, B_scan_number, Image_Type


,File_Path,BCVA,CST,Eye_ID,Patient_ID,Patient_ID_String,Week,Eye,Filename,B_scan_number,Image_Type
0,/Prime_FULL/01-001/W0/OS/27.png,97.0,275.0,58,58,01-001,0,OS,27.png,27,png
1,/Prime_FULL/01-001/W0/OS/0.png,97.0,275.0,58,58,01-001,0,OS,0.png,0,png
2,/Prime_FULL/01-001/W0/OS/1.png,97.0,275.0,58,58,01-001,0,OS,1.png,1,png
3,/Prime_FULL/01-001/W0/OS/10.png,97.0,275.0,58,58,01-001,0,OS,10.png,10,png
4,/Prime_FULL/01-001/W0/OS/11.png,97.0,275.0,58,58,01-001,0,OS,11.png,11,png
...,...,...,...,...,...,...,...,...,...,...,...
32332,/Prime_FULL/02-046/W8/OD/5.tif,91.0,272.0,96,96,02-046,8,OD,5.tif,5,tif
32333,/Prime_FULL/02-046/W8/OD/6.tif,91.0,272.0,96,96,02-046,8,OD,6.tif,6,tif
32334,/Prime_FULL/02-046/W8/OD/7.tif,91.0,272.0,96,96,02-046,8,OD,7.tif,7,tif
32335,/Prime_FULL/02-046/W8/OD/8.tif,91.0,272.0,96,96,02-046,8,OD,8.tif,8,tif


We have 21 time points from Week 0 to Week 104. Now let's see how many unique eyes have both baseline and outcome data

In [11]:
from pathlib import Path

# Get unique eyes at baseline and outcome
baseline_eyes = clinical_df[clinical_df['Week'] == 0]['Eye_ID'].unique()
outcome_eyes = clinical_df[clinical_df['Week'] == 104]['Eye_ID'].unique()

print(f"\nEyes with baseline (W0) data: {len(baseline_eyes)}")
print(f"Eyes with outcome (W104) data: {len(outcome_eyes)}")

# Find eyes with BOTH baseline and outcome
complete_eyes = set(baseline_eyes) & set(outcome_eyes)
print(f"\nEyes with BOTH W0 and W104: {len(complete_eyes)}")

# Eyes missing outcome
missing_outcome = set(baseline_eyes) - set(outcome_eyes)
if missing_outcome:
    print(f"\n⚠️ Eyes with baseline but NO outcome (W104): {len(missing_outcome)}")
    print(f"   Eye IDs: {sorted(list(missing_outcome))[:10]}...")

# Check image types at baseline
print(f"\n--- Image breakdown at baseline (W0) ---")
baseline_df = clinical_df[clinical_df['Week'] == 0].copy()
print(f"Total baseline images: {len(baseline_df)}")
print(f"Images per eye (average): {len(baseline_df) / len(baseline_eyes):.0f}")

# Look at file extensions
baseline_df['Extension'] = baseline_df['File_Path'].str.split('.').str[-1]
print(f"\nFile types at baseline:")
print(baseline_df['Extension'].value_counts())



Eyes with baseline (W0) data: 40
Eyes with outcome (W104) data: 23

Eyes with BOTH W0 and W104: 23

⚠️ Eyes with baseline but NO outcome (W104): 17
   Eye IDs: [np.int64(57), np.int64(59), np.int64(64), np.int64(68), np.int64(70), np.int64(74), np.int64(75), np.int64(77), np.int64(79), np.int64(81)]...

--- Image breakdown at baseline (W0) ---
Total baseline images: 1960
Images per eye (average): 49

File types at baseline:
Extension
tif    1911
png      49
Name: count, dtype: int64


⚠️Only 23 eyes (not 40) have complete baseline→outcome data. And we can see the image types:

⚠️ TIF 不是 OCT B-scans, png 不是 fundas images



In [12]:
print("="*80)
print("IMAGE TYPE DISTRIBUTION ANALYSIS")
print("="*80)

# Check image types at baseline for all eyes
baseline = clinical_df[clinical_df['Week'] == 0]

print(f"\nBaseline (W0) images: {len(baseline)}")
print(f"  TIF (OCT): {(baseline['Image_Type'] == 'tif').sum()}")
print(f"  PNG (Fundus? no because the data is wrong for Eye 58): {(baseline['Image_Type'] == 'png').sum()}")

# Check per eye: how many TIF vs PNG
baseline_summary = baseline.groupby('Eye_ID')['Image_Type'].value_counts().unstack(fill_value=0)
print(f"\nImage types per eye at baseline:")
print(baseline_summary)

# Specifically for Eye 58
print("\n" + "="*80)
print("EYE 58 DETAILED BREAKDOWN")
print("="*80)

eye_58_baseline = baseline[baseline['Eye_ID'] == 58]
print(f"\nEye 58 at baseline (W0):")
print(f"  PNG images: {(eye_58_baseline['Image_Type'] == 'png').sum()}")
print(f"  TIF images: {(eye_58_baseline['Image_Type'] == 'tif').sum()}")

# Check if fundus is separate from OCT B-scans
print("\n--- Hypothesis: Each eye has BOTH fundus (PNG) + OCT (TIF) ---")

# For each eye, count image types
eye_image_summary = baseline.groupby('Eye_ID').agg({
    'Image_Type': lambda x: dict(x.value_counts()),
    'File_Path': 'count'
}).rename(columns={'File_Path': 'Total_Images'})

print("\nFirst 10 eyes:")
print(eye_image_summary.head(10))

# Check if pattern is: 1 PNG (fundus) + 49 TIF (OCT)?
has_png = baseline.groupby('Eye_ID')['Image_Type'].apply(lambda x: (x == 'png').sum())
has_tif = baseline.groupby('Eye_ID')['Image_Type'].apply(lambda x: (x == 'tif').sum())

summary_df = pd.DataFrame({
    'PNG_count': has_png,
    'TIF_count': has_tif,
    'Total': has_png + has_tif
})

print("\n" + "="*80)
print("PATTERN CHECK")
print("="*80)
print(summary_df)

# Common pattern?
most_common_pattern = summary_df.value_counts().head(5)
print("\nMost common patterns:")
print(most_common_pattern)

IMAGE TYPE DISTRIBUTION ANALYSIS

Baseline (W0) images: 1960
  TIF (OCT): 1911
  PNG (Fundus? no because the data is wrong for Eye 58): 49

Image types per eye at baseline:
Image_Type  png  tif
Eye_ID              
57            0   49
58           49    0
59            0   49
60            0   49
61            0   49
62            0   49
63            0   49
64            0   49
65            0   49
66            0   49
67            0   49
68            0   49
69            0   49
70            0   49
71            0   49
72            0   49
73            0   49
74            0   49
75            0   49
76            0   49
77            0   49
78            0   49
79            0   49
80            0   49
81            0   49
82            0   49
83            0   49
84            0   49
85            0   49
86            0   49
87            0   49
88            0   49
89            0   49
90            0   49
91            0   49
92            0   49
93            0   49
94      

39 eyes have OCT scans (TIF) - 49 B-scans each

⚠ 1 eye has PNG images (Eye 58) - this is unusual, might be a different format or error

So we have:

23 eyes with complete baseline→outcome data

22 eyes have OCT (TIF) at baseline

1 eye has PNG at baseline (Eye 58)

In [13]:
# clinical_df是否包含fundus图像？

print("="*80)
print("CHECKING FOR FUNDUS IMAGES IN clinical_df")
print("="*80)

# Check if fundus images are in the dataframe
fundus_check = clinical_df['File_Path'].str.contains('fundus', case=False, na=False)
print(f"\nFundus images in clinical_df: {fundus_check.sum()}")

if fundus_check.sum() > 0:
    print("\n✓ Fundus images ARE included!")
    print("\nSample fundus paths:")
    print(clinical_df[fundus_check]['File_Path'].head(5).tolist())
else:
    print("\n⚠️ Fundus images are NOT in clinical_df")
    print("   They exist in the file system but weren't included in the CSV")

# Check filenames at baseline
print("\n" + "="*80)
print("FILENAME PATTERNS AT BASELINE")
print("="*80)

baseline = clinical_df[clinical_df['Week'] == 0]
baseline_filenames = baseline['Filename'].value_counts()

print(f"\nUnique filenames at baseline: {len(baseline_filenames)}")
print("\nTop 10 most common filenames:")
print(baseline_filenames.head(10))

# Check if we see "fundus" in any filename
fundus_files = baseline[baseline['Filename'].str.contains('fundus', case=False, na=False)]
print(f"\nFiles with 'fundus' in name: {len(fundus_files)}")

# Sample one patient to see structure
sample_patient = '01-020'
sample_data = baseline[baseline['Patient_ID_String'] == sample_patient]
print(f"\n" + "="*80)
print(f"SAMPLE: Patient {sample_patient} at baseline")
print("="*80)
print(f"Total images: {len(sample_data)}")
print("\nAll filenames:")
print(sorted(sample_data['Filename'].tolist()))

CHECKING FOR FUNDUS IMAGES IN clinical_df

Fundus images in clinical_df: 0

⚠️ Fundus images are NOT in clinical_df
   They exist in the file system but weren't included in the CSV

FILENAME PATTERNS AT BASELINE

Unique filenames at baseline: 98

Top 10 most common filenames:
Filename
4.tif     39
39.tif    39
23.tif    39
24.tif    39
25.tif    39
26.tif    39
28.tif    39
29.tif    39
3.tif     39
30.tif    39
Name: count, dtype: int64

Files with 'fundus' in name: 0

SAMPLE: Patient 01-020 at baseline
Total images: 49

All filenames:
['0.tif', '1.tif', '10.tif', '11.tif', '12.tif', '13.tif', '14.tif', '15.tif', '16.tif', '17.tif', '18.tif', '19.tif', '2.tif', '20.tif', '21.tif', '22.tif', '23.tif', '24.tif', '25.tif', '26.tif', '27.tif', '28.tif', '29.tif', '3.tif', '30.tif', '31.tif', '32.tif', '33.tif', '34.tif', '35.tif', '36.tif', '37.tif', '38.tif', '39.tif', '4.tif', '40.tif', '41.tif', '42.tif', '43.tif', '44.tif', '45.tif', '46.tif', '47.tif', '48.tif', '5.tif', '6.tif', '7.

In [14]:
# scan the file system for fundus images to verify their existence and gather path info

print("="*80)
print("STEP 1: SCANNING FILE SYSTEM FOR FUNDUS IMAGES")
print("="*80)

# 设置基础路径
BASE_DIR = Path(r"C:\Users\ronny\Desktop\Thesis\Prime data extracted\Prime_FULL")
BASE_PREFIX = r"C:\Users\ronny\Desktop\Thesis\Prime data extracted"

print(f"\nScanning directory: {BASE_DIR}")
print("Looking for fundus images...")

# 扫描所有fundus图像
fundus_files = []

# 遍历所有患者文件夹
for patient_dir in BASE_DIR.iterdir():
    if not patient_dir.is_dir():
        continue
    
    patient_id = patient_dir.name  # e.g., "01-001"
    
    # 遍历所有week文件夹
    for week_dir in patient_dir.iterdir():
        if not week_dir.is_dir() or not week_dir.name.startswith('W'):
            continue
        
        week_match = re.search(r'W(\d+)', week_dir.name)
        if not week_match:
            continue
        week = int(week_match.group(1))
        
        # 遍历所有eye文件夹 (OD/OS)
        for eye_dir in week_dir.iterdir():
            if not eye_dir.is_dir() or eye_dir.name not in ['OD', 'OS']:
                continue
            
            eye = eye_dir.name
            
            # 查找fundus文件 (可能是png或tif)
            for fundus_file in eye_dir.glob('fundus_W*.png'):
                # 转换为相对路径，移除BASE_PREFIX
                full_path = str(fundus_file).replace('\\', '/')
                relative_path = full_path.replace(BASE_PREFIX.replace('\\', '/'), '').lstrip('/')
                
                fundus_files.append({
                    'Patient_ID_String': patient_id,
                    'Week': week,
                    'Eye': eye,
                    'File_Path': relative_path,  # 相对路径
                    'Filename': fundus_file.name,
                    'Extension': fundus_file.suffix[1:]  # Remove the dot
                })
            
            for fundus_file in eye_dir.glob('fundus_W*.tif'):
                # 转换为相对路径，移除BASE_PREFIX
                full_path = str(fundus_file).replace('\\', '/')
                relative_path = full_path.replace(BASE_PREFIX.replace('\\', '/'), '').lstrip('/')
                
                fundus_files.append({
                    'Patient_ID_String': patient_id,
                    'Week': week,
                    'Eye': eye,
                    'File_Path': relative_path,  # 相对路径
                    'Filename': fundus_file.name,
                    'Extension': fundus_file.suffix[1:]
                })

# 创建DataFrame
fundus_scanned_df = pd.DataFrame(fundus_files)

print(f"\n✓ Found {len(fundus_scanned_df)} fundus images!")

if len(fundus_scanned_df) > 0:
    print(f"\n--- Extension distribution ---")
    print(fundus_scanned_df['Extension'].value_counts())
    
    print(f"\n--- Sample fundus files found (with relative paths) ---")
    print(fundus_scanned_df.head(10).to_string(index=False))
    
    print(f"\n--- Statistics ---")
    print(f"Unique patients: {fundus_scanned_df['Patient_ID_String'].nunique()}")
    print(f"Unique weeks: {sorted(fundus_scanned_df['Week'].unique())}")
    print(f"Eyes: {dict(fundus_scanned_df['Eye'].value_counts())}")
else:
    print("\n⚠️ No fundus images found! Please check the BASE_DIR path.")

# 保存扫描结果
fundus_scanned_df.to_csv('fundus_scanned_files.csv', index=False)


STEP 1: SCANNING FILE SYSTEM FOR FUNDUS IMAGES

Scanning directory: C:\Users\ronny\Desktop\Thesis\Prime data extracted\Prime_FULL
Looking for fundus images...

✓ Found 1284 fundus images!

--- Extension distribution ---
Extension
tif    1270
png      14
Name: count, dtype: int64

--- Sample fundus files found (with relative paths) ---
Patient_ID_String  Week Eye                                 File_Path        Filename Extension
           01-001     0  OD     Prime_FULL/01-001/W0/OD/fundus_W0.png   fundus_W0.png       png
           01-001     0  OS     Prime_FULL/01-001/W0/OS/fundus_W0.png   fundus_W0.png       png
           01-001   100  OD Prime_FULL/01-001/W100/OD/fundus_W100.tif fundus_W100.tif       tif
           01-001   100  OS Prime_FULL/01-001/W100/OS/fundus_W100.tif fundus_W100.tif       tif
           01-001   104  OD Prime_FULL/01-001/W104/OD/fundus_W104.tif fundus_W104.tif       tif
           01-001   104  OS Prime_FULL/01-001/W104/OS/fundus_W104.tif fundus_W104.tif  

## merge demo_df with clinical_df on Patient ID

In [15]:
print('demo_df cols:', demo_df.columns.tolist())
demo_df

demo_df cols: ['Patient \nID', 'Treatment Arm', 'Study\n Eye', 'Age', 'Gender', 'Ethnicity', 'Race', 'Type of\n Diabetes', 'Number of Years with Diabetes', 'Baseline HbA1c', 'W24 HbA1c', 'W52 HbA1c', 'W76 HbA1c', 'W104 HbA1c', 'BMI (kg/m^2)', 'ETDRS BCVA', 'CST', 'Injection', 'DRSS', 'Leakage Index']


,Patient \nID,Treatment Arm,Study\n Eye,Age,Gender,Ethnicity,Race,Type of\n Diabetes,Number of Years with Diabetes,Baseline HbA1c,W24 HbA1c,W52 HbA1c,W76 HbA1c,W104 HbA1c,BMI (kg/m^2),ETDRS BCVA,CST,Injection,DRSS,Leakage Index
0,01-001,2,OS,44,M,N H/L,White,2,20,7.1,8.7,8.4,9.1,8.4,NaN,97,275,Yes,53,1.59
1,01-002,2,OD,56,F,N H/L,White,2,25,11.3,9.1,,NaN,NaN,34.484657,68,238,Yes,47,3.56
2,01-013,2,OD,38,M,H/L,White,1,13,8.0,9.5,9.6,11.7,9.8,25.997929,88,303,Yes,53,5.08
3,01-014,2,OS,55,M,N H/L,White,2,12,10.1,7.2,7.4,7.4,7.4,31.871377,95,256,Yes,53,2.34
4,01-023,2,OD,56,M,H/L,White,2,22,7.1,6.7,5.9,6.9,NaN,35.669938,81,267,Yes,61,4.67
5,01-027,2,OS,65,F,H/L,White,2,6,7.0,8.1,9.2,9.8,10.9,26.751734,89,292,Yes,61,6.26
6,01-028,2,OS,46,M,N H/L,Black,2,21,8.1,8.2,9.8,10.9,NaN,18.602501,59,287,Yes,71,4.16
7,01-035,2,OD,52,M,H/L,White,2,29,12.8,10.9,11.9,11.3,NaN,29.164344,81,240,Yes,65,1.13
8,01-038,2,OS,34,F,H/L,White,2,18,12.2,11.0,12.5,13.0,10.5,33.298467,80,244,Yes,53,6.42
9,01-047,2,OD,55,M,H/L,White,2,10,9.5,7.4,8.5,7.5,9.1,33.470767,94,278,Yes,61,3.43


In [16]:
print('clinical_df cols:', clinical_df.columns.tolist())
clinical_df

clinical_df cols: ['File_Path', 'BCVA', 'CST', 'Eye_ID', 'Patient_ID', 'Patient_ID_String', 'Week', 'Eye', 'Filename', 'B_scan_number', 'Image_Type']


,File_Path,BCVA,CST,Eye_ID,Patient_ID,Patient_ID_String,Week,Eye,Filename,B_scan_number,Image_Type
0,/Prime_FULL/01-001/W0/OS/27.png,97.0,275.0,58,58,01-001,0,OS,27.png,27,png
1,/Prime_FULL/01-001/W0/OS/0.png,97.0,275.0,58,58,01-001,0,OS,0.png,0,png
2,/Prime_FULL/01-001/W0/OS/1.png,97.0,275.0,58,58,01-001,0,OS,1.png,1,png
3,/Prime_FULL/01-001/W0/OS/10.png,97.0,275.0,58,58,01-001,0,OS,10.png,10,png
4,/Prime_FULL/01-001/W0/OS/11.png,97.0,275.0,58,58,01-001,0,OS,11.png,11,png
...,...,...,...,...,...,...,...,...,...,...,...
32332,/Prime_FULL/02-046/W8/OD/5.tif,91.0,272.0,96,96,02-046,8,OD,5.tif,5,tif
32333,/Prime_FULL/02-046/W8/OD/6.tif,91.0,272.0,96,96,02-046,8,OD,6.tif,6,tif
32334,/Prime_FULL/02-046/W8/OD/7.tif,91.0,272.0,96,96,02-046,8,OD,7.tif,7,tif
32335,/Prime_FULL/02-046/W8/OD/8.tif,91.0,272.0,96,96,02-046,8,OD,8.tif,8,tif


In [17]:
print('fundus_scanned_df cols:', fundus_scanned_df.columns.tolist())
fundus_scanned_df

fundus_scanned_df cols: ['Patient_ID_String', 'Week', 'Eye', 'File_Path', 'Filename', 'Extension']


,Patient_ID_String,Week,Eye,File_Path,Filename,Extension
0,01-001,0,OD,Prime_FULL/01-001/W0/OD/fundus_W0.png,fundus_W0.png,png
1,01-001,0,OS,Prime_FULL/01-001/W0/OS/fundus_W0.png,fundus_W0.png,png
2,01-001,100,OD,Prime_FULL/01-001/W100/OD/fundus_W100.tif,fundus_W100.tif,tif
3,01-001,100,OS,Prime_FULL/01-001/W100/OS/fundus_W100.tif,fundus_W100.tif,tif
4,01-001,104,OD,Prime_FULL/01-001/W104/OD/fundus_W104.tif,fundus_W104.tif,tif
...,...,...,...,...,...,...
1279,02-046,52,OS,Prime_FULL/02-046/W52/OS/fundus_W52.tif,fundus_W52.tif,tif
1280,02-046,60,OD,Prime_FULL/02-046/W60/OD/fundus_W60.tif,fundus_W60.tif,tif
1281,02-046,60,OS,Prime_FULL/02-046/W60/OS/fundus_W60.tif,fundus_W60.tif,tif
1282,02-046,8,OD,Prime_FULL/02-046/W8/OD/fundus_W8.tif,fundus_W8.tif,tif


In [18]:
print("="*80)
print("MERGING: ADDING FUNDUS AS NEW ROWS")
print("="*80)

# Step 1: clinical_df是基础（32,337张OCT）
print(f"Base: clinical_df with {len(clinical_df):,} OCT images")

# Step 2: 准备fundus数据
fundus_to_add = fundus_scanned_df.copy()

# 为fundus添加标签（从同一(Patient, Week, Eye)的OCT借用）
oct_labels = clinical_df.groupby(['Patient_ID_String', 'Week', 'Eye']).agg({
    'BCVA': 'first',
    'CST': 'first', 
    'Eye_ID': 'first',
    'Patient_ID': 'first'
}).reset_index()

fundus_with_labels = fundus_to_add.merge(
    oct_labels,
    on=['Patient_ID_String', 'Week', 'Eye'],
    how='left'
)

# 添加缺失的列
fundus_with_labels['Image_Type'] = 'fundus'
fundus_with_labels['B_scan_number'] = None

# 匹配列结构
for col in clinical_df.columns:
    if col not in fundus_with_labels.columns:
        fundus_with_labels[col] = None

fundus_with_labels = fundus_with_labels[clinical_df.columns]

# Step 3: 合并（UNION）
clinical_with_fundus = pd.concat([clinical_df, fundus_with_labels], ignore_index=True)

print(f"\n✓ Combined:")
print(f"  OCT images: {len(clinical_df):,}")
print(f"  Fundus images: {len(fundus_with_labels):,}")
print(f"  Total: {len(clinical_with_fundus):,}")

# Step 4: LEFT JOIN demographics (用列索引，不rename)
print("\n" + "="*80)
print("MERGING WITH DEMOGRAPHICS")
print("="*80)

# 直接用索引访问列
demo_for_merge = demo_df.iloc[:, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]].copy()

# 给列起新名字（直接赋值）
demo_for_merge.columns = [
    'Patient_ID_String',    # 0
    'Treatment_Arm',        # 1
    'Study_Eye',            # 2
    'Age',                  # 3
    'Gender',               # 4
    'Ethnicity',            # 5
    'Race',                 # 6
    'Diabetes_Type',        # 7
    'Years_Diabetes',       # 8
    'HbA1c_Baseline',       # 9
    'HbA1c_W24',            # 10
    'HbA1c_W52',            # 11
    'HbA1c_W76',            # 12
    'HbA1c_W104',           # 13
    'BMI',                  # 14
    'BCVA_Demo',            # 15
    'CST_Demo',             # 16
    'Injection_Baseline',   # 17
    'DRSS_Baseline',        # 18
    'Leakage_Index_Baseline' # 19
]

print(f"Demo columns: {demo_for_merge.columns.tolist()}")

# 合并
clinical_full = clinical_with_fundus.merge(
    demo_for_merge,
    on='Patient_ID_String',
    how='left'
)

print(f"\n✓ Final merged dataset: {clinical_full.shape}")
print(f"Rows with demographics: {clinical_full['Treatment_Arm'].notna().sum():,}/{len(clinical_full):,}")

# 验证
baseline = clinical_full[clinical_full['Week'] == 0]
print(f"\n--- Baseline verification ---")
print(f"Total baseline images: {len(baseline):,}")
print(baseline['Image_Type'].value_counts())

# 检查一个eye
sample_eye = baseline['Eye_ID'].iloc[0]
sample_data = baseline[baseline['Eye_ID'] == sample_eye]
print(f"\nSample Eye {sample_eye}:")
print(f"  Total images: {len(sample_data)}")
print(f"  Image types: {dict(sample_data['Image_Type'].value_counts())}")

# 保存
clinical_full.to_csv('clinical_full_merged.csv', index=False)
print(f"\n✓ Saved to 'clinical_full_merged.csv'")


MERGING: ADDING FUNDUS AS NEW ROWS
Base: clinical_df with 32,337 OCT images

✓ Combined:
  OCT images: 32,337
  Fundus images: 1,284
  Total: 33,621

MERGING WITH DEMOGRAPHICS
Demo columns: ['Patient_ID_String', 'Treatment_Arm', 'Study_Eye', 'Age', 'Gender', 'Ethnicity', 'Race', 'Diabetes_Type', 'Years_Diabetes', 'HbA1c_Baseline', 'HbA1c_W24', 'HbA1c_W52', 'HbA1c_W76', 'HbA1c_W104', 'BMI', 'BCVA_Demo', 'CST_Demo', 'Injection_Baseline', 'DRSS_Baseline', 'Leakage_Index_Baseline']

✓ Final merged dataset: (33621, 30)
Rows with demographics: 33,621/33,621

--- Baseline verification ---
Total baseline images: 2,036
Image_Type
tif       1911
fundus      76
png         49
Name: count, dtype: int64

Sample Eye 58.0:
  Total images: 50
  Image types: {'png': np.int64(49), 'fundus': np.int64(1)}

✓ Saved to 'clinical_full_merged.csv'


In [19]:
clinical_full

,File_Path,BCVA,CST,Eye_ID,Patient_ID,Patient_ID_String,Week,Eye,Filename,B_scan_number,...,HbA1c_W24,HbA1c_W52,HbA1c_W76,HbA1c_W104,BMI,BCVA_Demo,CST_Demo,Injection_Baseline,DRSS_Baseline,Leakage_Index_Baseline
0,/Prime_FULL/01-001/W0/OS/27.png,97.0,275.0,58.0,58.0,01-001,0,OS,27.png,27,...,8.7,8.4,9.1,8.4,NaN,97,275,Yes,53,1.59
1,/Prime_FULL/01-001/W0/OS/0.png,97.0,275.0,58.0,58.0,01-001,0,OS,0.png,0,...,8.7,8.4,9.1,8.4,NaN,97,275,Yes,53,1.59
2,/Prime_FULL/01-001/W0/OS/1.png,97.0,275.0,58.0,58.0,01-001,0,OS,1.png,1,...,8.7,8.4,9.1,8.4,NaN,97,275,Yes,53,1.59
3,/Prime_FULL/01-001/W0/OS/10.png,97.0,275.0,58.0,58.0,01-001,0,OS,10.png,10,...,8.7,8.4,9.1,8.4,NaN,97,275,Yes,53,1.59
4,/Prime_FULL/01-001/W0/OS/11.png,97.0,275.0,58.0,58.0,01-001,0,OS,11.png,11,...,8.7,8.4,9.1,8.4,NaN,97,275,Yes,53,1.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33616,Prime_FULL/02-046/W52/OS/fundus_W52.tif,NaN,NaN,NaN,NaN,02-046,52,OS,fundus_W52.tif,None,...,7.8,8.3,NaN,8.1,33.718616,89,283,Yes,71,6.39
33617,Prime_FULL/02-046/W60/OD/fundus_W60.tif,89.0,242.0,96.0,96.0,02-046,60,OD,fundus_W60.tif,None,...,7.8,8.3,NaN,8.1,33.718616,89,283,Yes,71,6.39
33618,Prime_FULL/02-046/W60/OS/fundus_W60.tif,NaN,NaN,NaN,NaN,02-046,60,OS,fundus_W60.tif,None,...,7.8,8.3,NaN,8.1,33.718616,89,283,Yes,71,6.39
33619,Prime_FULL/02-046/W8/OD/fundus_W8.tif,91.0,272.0,96.0,96.0,02-046,8,OD,fundus_W8.tif,None,...,7.8,8.3,NaN,8.1,33.718616,89,283,Yes,71,6.39


In [20]:
print("="*80)
print("DIAGNOSING: MISSING EYE DATA")
print("="*80)

# 检查clinical_df中的眼睛分布
print("Clinical_df eye distribution:")
print(clinical_df.groupby(['Patient_ID_String', 'Eye'])['Eye_ID'].nunique())

# 检查fundus中的眼睛分布
print("\nFundus eye distribution:")
fundus_eyes = fundus_scanned_df.groupby(['Patient_ID_String', 'Eye']).size()
print(f"Total combinations: {len(fundus_eyes)}")

# 找出fundus有但clinical_df没有的(Patient, Eye)组合
clinical_combos = set(zip(clinical_df['Patient_ID_String'], clinical_df['Eye']))
fundus_combos = set(zip(fundus_scanned_df['Patient_ID_String'], fundus_scanned_df['Eye']))

missing_in_clinical = fundus_combos - clinical_combos

print(f"\n⚠️ Fundus has data for {len(fundus_combos)} (Patient, Eye) combinations")
print(f"⚠️ Clinical has data for {len(clinical_combos)} (Patient, Eye) combinations")
print(f"⚠️ Missing in clinical: {len(missing_in_clinical)}")

if missing_in_clinical:
    print("\nSample missing combinations:")
    for combo in list(missing_in_clinical)[:5]:
        print(f"  Patient {combo[0]}, Eye {combo[1]}")

# 检查demographics中的study eye
print("\n--- Demographics Study Eye ---")
print(demo_df.iloc[:, 2].value_counts())  # Study Eye列

# 推断问题
print("\n--- HYPOTHESIS ---")
print("Fundus images exist for BOTH eyes (OD and OS)")
print("But clinical_df only has data for the 'Study Eye' from demographics")
print("\nOptions:")
print("1. Only use Study Eye data (discard other eye's fundus)")
print("2. Use other eye's data but mark BCVA/CST as missing")
print("3. Assume both eyes have same BCVA/CST (duplicate)")

DIAGNOSING: MISSING EYE DATA
Clinical_df eye distribution:
Patient_ID_String  Eye
01-001             OS     1
01-002             OD     1
01-012             OD     1
01-013             OD     1
01-014             OS     1
01-020             OS     1
01-023             OD     1
01-025             OD     1
01-026             OD     1
01-027             OS     1
01-028             OS     1
01-035             OD     1
01-037             OS     1
01-038             OS     1
01-040             OD     1
01-047             OD     1
01-048             OD     1
02-004             OD     1
02-005             OS     1
02-008             OD     1
02-010             OD     1
02-015             OD     1
02-016             OS     1
02-017             OD     1
02-018             OD     1
02-019             OS     1
02-024             OD     1
02-029             OS     1
02-030             OD     1
02-031             OD     1
02-032             OS     1
02-034             OD     1
02-036             OD 

问题确认：

📁 文件系统：两只眼都有fundus图像（79个组合 ≈ 40患者 × 2眼）
📊 Clinical CSV：只有Study Eye的数据（40个组合）
❌ 39个(Patient, Eye)组合有fundus但没有BCVA/CST标签

In [21]:
print("="*80)
print("SOLUTION: FILTER TO STUDY EYE ONLY")
print("="*80)

# Step 1: 提取每个患者的Study Eye
demo_for_merge = demo_df.iloc[:, [0, 2]].copy()  # Patient ID and Study Eye
demo_for_merge.columns = ['Patient_ID_String', 'Study_Eye']

print("Study Eye mapping:")
print(demo_for_merge.head(10))

# Step 2: 过滤fundus，只保留Study Eye
fundus_filtered = fundus_scanned_df.merge(
    demo_for_merge,
    on='Patient_ID_String',
    how='left'
)

# 只保留匹配Study Eye的fundus
fundus_filtered = fundus_filtered[fundus_filtered['Eye'] == fundus_filtered['Study_Eye']].copy()
fundus_filtered = fundus_filtered.drop('Study_Eye', axis=1)

print(f"\n✓ Fundus filtered:")
print(f"  Before: {len(fundus_scanned_df)} fundus images (both eyes)")
print(f"  After: {len(fundus_filtered)} fundus images (study eye only)")

# Step 3: 统一路径格式（添加开头的 /）
fundus_filtered['File_Path'] = '/' + fundus_filtered['File_Path']

print(f"\n✓ Path format unified:")
print("Sample paths:")
print(fundus_filtered['File_Path'].head(3).tolist())

# Step 4: 为fundus添加标签
oct_labels = clinical_df.groupby(['Patient_ID_String', 'Week', 'Eye']).agg({
    'BCVA': 'first',
    'CST': 'first', 
    'Eye_ID': 'first',
    'Patient_ID': 'first'
}).reset_index()

fundus_with_labels = fundus_filtered.merge(
    oct_labels,
    on=['Patient_ID_String', 'Week', 'Eye'],
    how='left'
)

print(f"\n✓ Fundus with labels:")
print(f"  Total: {len(fundus_with_labels)}")
print(f"  Missing BCVA: {fundus_with_labels['BCVA'].isna().sum()}")
print(f"  Missing Eye_ID: {fundus_with_labels['Eye_ID'].isna().sum()}")

# 添加Image_Type
fundus_with_labels['Image_Type'] = 'fundus'
fundus_with_labels['B_scan_number'] = None

# 匹配列
for col in clinical_df.columns:
    if col not in fundus_with_labels.columns:
        fundus_with_labels[col] = None

fundus_with_labels = fundus_with_labels[clinical_df.columns]

# Step 5: 合并OCT + Fundus
clinical_with_fundus = pd.concat([clinical_df, fundus_with_labels], ignore_index=True)

print(f"\n✓ Combined dataset:")
print(f"  OCT: {len(clinical_df):,}")
print(f"  Fundus: {len(fundus_with_labels):,}")
print(f"  Total: {len(clinical_with_fundus):,}")

# Step 6: 合并demographics
demo_full = demo_df.iloc[:, :].copy()
demo_full.columns = [
    'Patient_ID_String', 'Treatment_Arm', 'Study_Eye', 'Age', 'Gender',
    'Ethnicity', 'Race', 'Diabetes_Type', 'Years_Diabetes',
    'HbA1c_Baseline', 'HbA1c_W24', 'HbA1c_W52', 'HbA1c_W76', 'HbA1c_W104',
    'BMI', 'BCVA_Demo', 'CST_Demo', 'Injection_Baseline',
    'DRSS_Baseline', 'Leakage_Index_Baseline'
]

clinical_full = clinical_with_fundus.merge(
    demo_full,
    on='Patient_ID_String',
    how='left'
)

print(f"\n✓ Final dataset: {clinical_full.shape}")

# 验证
baseline = clinical_full[clinical_full['Week'] == 0]
print(f"\n--- Baseline Summary ---")
print(f"Total images: {len(baseline):,}")
print(baseline['Image_Type'].value_counts())

# 每个eye应该有50张图
eye_counts = baseline.groupby('Eye_ID').size()
print(f"\nImages per eye: min={eye_counts.min()}, max={eye_counts.max()}, mean={eye_counts.mean():.1f}")

# 保存
clinical_full.to_csv('clinical_full_merged2.csv', index=False)
print(f"\n✓ Saved!")

print("\n" + "="*80)
print("COMPLETE!")
print("="*80)
print(f"Final dataset uses STUDY EYE ONLY")
print(f"  Patients: 40")
print(f"  Eyes: 40 (one per patient)")
print(f"  Images per visit per eye: ~50 (49 OCT + 1 fundus)")

SOLUTION: FILTER TO STUDY EYE ONLY
Study Eye mapping:
  Patient_ID_String Study_Eye
0            01-001        OS
1            01-002        OD
2            01-013        OD
3            01-014        OS
4            01-023        OD
5            01-027        OS
6            01-028        OS
7            01-035        OD
8            01-038        OS
9            01-047        OD

✓ Fundus filtered:
  Before: 1284 fundus images (both eyes)
  After: 660 fundus images (study eye only)

✓ Path format unified:
Sample paths:
['/Prime_FULL/01-001/W0/OS/fundus_W0.png', '/Prime_FULL/01-001/W100/OS/fundus_W100.tif', '/Prime_FULL/01-001/W104/OS/fundus_W104.tif']

✓ Fundus with labels:
  Total: 660
  Missing BCVA: 0
  Missing Eye_ID: 0

✓ Combined dataset:
  OCT: 32,337
  Fundus: 660
  Total: 32,997

✓ Final dataset: (32997, 30)

--- Baseline Summary ---
Total images: 2,000
Image_Type
tif       1911
png         49
fundus      40
Name: count, dtype: int64

Images per eye: min=50, max=50, mean=50.

In [22]:
# Check missing values in clinical_full
missing_summary = clinical_full.isnull().sum()
print("\n--- Missing Values Summary ---")
print(missing_summary)


--- Missing Values Summary ---
File_Path                     0
BCVA                          0
CST                           0
Eye_ID                        0
Patient_ID                    0
Patient_ID_String             0
Week                          0
Eye                           0
Filename                      0
B_scan_number               660
Image_Type                    0
Treatment_Arm                 0
Study_Eye                     0
Age                           0
Gender                        0
Ethnicity                     0
Race                          0
Diabetes_Type                 0
Years_Diabetes                0
HbA1c_Baseline                0
HbA1c_W24                  7500
HbA1c_W52                  6600
HbA1c_W76                 13947
HbA1c_W104                12497
BMI                        1000
BCVA_Demo                     0
CST_Demo                      0
Injection_Baseline            0
DRSS_Baseline                 0
Leakage_Index_Baseline        0
dtype: i

✅ 正常的缺失值
1. B_scan_number: 660 missing

✅ 这是对的！
Fundus图像没有B-scan编号（只有OCT才有0-48的编号）
660个缺失 = 660个fundus图像
完全符合预期

2. HbA1c at follow-up visits

HbA1c_W24: 7,500 missing
HbA1c_W52: 6,600 missing
HbA1c_W76: 13,947 missing
HbA1c_W104: 12,497 missing

✅ 这也正常！

HbA1c是患者级别的测量，不是每次visit都测
Demographics只有40个患者的HbA1c数据
但clinical_full有32,997行（每张图像一行）
所以HbA1c会在很多行重复或缺失
不影响建模 - 你会按Eye或Patient聚合

3. BMI: 1,000 missing

✅ Demographics中1个患者的BMI缺失
这1个患者的所有图像（约1,000张）都会缺失BMI
可以接受 - 只有1个患者

In [23]:
print("="*80)
print("CLINICAL_FULL DATASET SUMMARY")
print("="*80)

print("\n📊 OVERALL STATISTICS")
print("-" * 80)
print(f"Total images: {len(clinical_full):,}")
print(f"Total columns: {len(clinical_full.columns)}")
print(f"Dataset shape: {clinical_full.shape}")

print("\n👥 PATIENT & EYE LEVEL")
print("-" * 80)
print(f"Unique patients: {clinical_full['Patient_ID_String'].nunique()}")
print(f"Unique eyes (Eye_ID): {clinical_full['Eye_ID'].nunique()}")
print(f"Eyes per patient: {clinical_full['Eye_ID'].nunique() / clinical_full['Patient_ID_String'].nunique():.1f}")
print(f"\nStudy Eye distribution:")
print(clinical_full.groupby('Patient_ID_String')['Study_Eye'].first().value_counts())

print("\n📅 TEMPORAL COVERAGE")
print("-" * 80)
print(f"Time span: Week {int(clinical_full['Week'].min())} to Week {int(clinical_full['Week'].max())}")
print(f"Number of visit weeks: {clinical_full['Week'].nunique()}")
print(f"Visit weeks: {sorted(clinical_full['Week'].unique())}")

# Images per visit
images_per_week = clinical_full.groupby('Week').size()
print(f"\nImages per visit (average): {images_per_week.mean():.0f}")
print(f"Range: {images_per_week.min()} to {images_per_week.max()}")

print("\n🖼️ IMAGE BREAKDOWN")
print("-" * 80)
print(f"Image type distribution:")
print(clinical_full['Image_Type'].value_counts())
print(f"\nPercentage:")
for img_type, count in clinical_full['Image_Type'].value_counts().items():
    pct = count / len(clinical_full) * 100
    print(f"  {img_type}: {pct:.1f}%")

print("\n📸 BASELINE (Week 0) DETAIL")
print("-" * 80)
baseline = clinical_full[clinical_full['Week'] == 0]
print(f"Total baseline images: {len(baseline):,}")
print(f"Unique eyes at baseline: {baseline['Eye_ID'].nunique()}")
print(f"Images per eye at baseline: {len(baseline) / baseline['Eye_ID'].nunique():.0f}")
print(f"\nBaseline image types:")
print(baseline['Image_Type'].value_counts())

print("\n📸 OUTCOME (Week 104) DETAIL")
print("-" * 80)
outcome = clinical_full[clinical_full['Week'] == 104]
print(f"Total outcome images: {len(outcome):,}")
print(f"Unique eyes at outcome: {outcome['Eye_ID'].nunique()}")
print(f"Eyes with complete baseline→outcome: {len(set(baseline['Eye_ID']) & set(outcome['Eye_ID']))}")

print("\n🏥 CLINICAL MEASUREMENTS")
print("-" * 80)
print(f"BCVA range: {clinical_full['BCVA'].min():.0f} to {clinical_full['BCVA'].max():.0f} letters")
print(f"BCVA mean: {clinical_full['BCVA'].mean():.1f} ± {clinical_full['BCVA'].std():.1f}")
print(f"\nCST range: {clinical_full['CST'].min():.0f} to {clinical_full['CST'].max():.0f} μm")
print(f"CST mean: {clinical_full['CST'].mean():.0f} ± {clinical_full['CST'].std():.0f}")

print("\n👤 DEMOGRAPHICS (Patient-level)")
print("-" * 80)
# Get unique patient demographics
patient_demo = clinical_full.groupby('Patient_ID_String').first()
print(f"Age: {patient_demo['Age'].mean():.1f} ± {patient_demo['Age'].std():.1f} years")
print(f"  Range: {patient_demo['Age'].min():.0f} to {patient_demo['Age'].max():.0f}")
print(f"\nGender distribution:")
print(patient_demo['Gender'].value_counts())
print(f"\nTreatment Arm distribution:")
print(patient_demo['Treatment_Arm'].value_counts())
print(f"\nBaseline HbA1c: {patient_demo['HbA1c_Baseline'].mean():.2f} ± {patient_demo['HbA1c_Baseline'].std():.2f}%")
print(f"Baseline DRSS: {patient_demo['DRSS_Baseline'].mean():.1f} ± {patient_demo['DRSS_Baseline'].std():.1f}")

print("\n📁 FILE STRUCTURE")
print("-" * 80)
print(f"File extensions:")
extension_counts = clinical_full['Filename'].str.split('.').str[-1].value_counts()
print(extension_counts)

print("\n💾 DATA COMPLETENESS")
print("-" * 80)
key_columns = ['BCVA', 'CST', 'Age', 'Gender', 'HbA1c_Baseline', 'Treatment_Arm']
for col in key_columns:
    missing = clinical_full[col].isna().sum()
    pct = missing / len(clinical_full) * 100
    print(f"{col:20s}: {len(clinical_full) - missing:6,} complete ({100-pct:5.1f}%), {missing:5,} missing ({pct:4.1f}%)")

print("\n" + "="*80)
print("SUMMARY FOR THESIS")
print("="*80)
print(f"""
Dataset: PRIME Diabetic Retinopathy Study
- Sample size: {clinical_full['Patient_ID_String'].nunique()} patients, {clinical_full['Eye_ID'].nunique()} study eyes
- Study duration: {int(clinical_full['Week'].max())} weeks (2 years)
- Visit frequency: {clinical_full['Week'].nunique()} time points
- Total images: {len(clinical_full):,} 
  • OCT B-scans: {(clinical_full['Image_Type'].str.contains('oct')).sum():,} ({(clinical_full['Image_Type'].str.contains('oct')).sum()/len(clinical_full)*100:.1f}%)
  • Fundus images: {(clinical_full['Image_Type'] == 'fundus').sum():,} ({(clinical_full['Image_Type'] == 'fundus').sum()/len(clinical_full)*100:.1f}%)
- Images per eye per visit: ~50 (49 OCT + 1 fundus)
- Complete baseline→outcome data: {len(set(baseline['Eye_ID']) & set(outcome['Eye_ID']))} eyes
- Patient characteristics:
  • Age: {patient_demo['Age'].mean():.1f} ± {patient_demo['Age'].std():.1f} years
  • Gender: {(patient_demo['Gender']=='M').sum()}M / {(patient_demo['Gender']=='F').sum()}F
  • Baseline HbA1c: {patient_demo['HbA1c_Baseline'].mean():.1f}% ± {patient_demo['HbA1c_Baseline'].std():.1f}%
""")

CLINICAL_FULL DATASET SUMMARY

📊 OVERALL STATISTICS
--------------------------------------------------------------------------------
Total images: 32,997
Total columns: 30
Dataset shape: (32997, 30)

👥 PATIENT & EYE LEVEL
--------------------------------------------------------------------------------
Unique patients: 40
Unique eyes (Eye_ID): 40
Eyes per patient: 1.0

Study Eye distribution:
Study_Eye
OD    24
OS    16
Name: count, dtype: int64

📅 TEMPORAL COVERAGE
--------------------------------------------------------------------------------
Time span: Week 0 to Week 104
Number of visit weeks: 21
Visit weeks: [np.int64(0), np.int64(4), np.int64(8), np.int64(12), np.int64(16), np.int64(20), np.int64(24), np.int64(28), np.int64(32), np.int64(36), np.int64(40), np.int64(44), np.int64(48), np.int64(52), np.int64(60), np.int64(68), np.int64(76), np.int64(84), np.int64(92), np.int64(100), np.int64(104)]

Images per visit (average): 1571
Range: 900 to 2000

🖼️ IMAGE BREAKDOWN
-------------

# Biomarker CSV

In [26]:
biomarker_df = pd.read_csv("PrimeDR Biomarker Clinical Data Images.csv")
print('Biomaker cols:', biomarker_df.columns.tolist())
biomarker_df

Biomaker cols: ['Path (Trial/Arm/Folder/Visit/Eye/Image Name)', 'Scan (n/49)', 'Atrophy / thinning of retinal layers', 'Disruption of EZ', 'DRIL', 'IR hemorrhages', 'IR HRF', 'Partially attached vitreous face', 'Fully attached vitreous face', 'Preretinal tissue/hemorrhage', 'Vitreous debris', 'VMT', 'DRT/ME', 'Fluid (IRF)', 'Fluid (SRF)', 'Disruption of RPE', 'PED (serous)', 'SHRM', 'Eye_ID', 'BCVA', 'CST', 'Patient_ID']


,Path (Trial/Arm/Folder/Visit/Eye/Image Name),Scan (n/49),Atrophy / thinning of retinal layers,Disruption of EZ,DRIL,IR hemorrhages,IR HRF,Partially attached vitreous face,Fully attached vitreous face,Preretinal tissue/hemorrhage,...,DRT/ME,Fluid (IRF),Fluid (SRF),Disruption of RPE,PED (serous),SHRM,Eye_ID,BCVA,CST,Patient_ID
0,/Prime_FULL/02-010/W0/OD/0.tif,1,0,0,0,0,1,0.0,0.0,0.0,...,0,0,0,0,0,0,57,88,307,57
1,/Prime_FULL/02-010/W0/OD/1.tif,2,0,0,0,0,1,0.0,0.0,0.0,...,0,0,0,0,0,0,57,88,307,57
2,/Prime_FULL/02-010/W0/OD/2.tif,3,0,0,0,0,1,0.0,0.0,0.0,...,0,0,0,0,0,0,57,88,307,57
3,/Prime_FULL/02-010/W0/OD/3.tif,4,0,0,0,0,1,0.0,0.0,0.0,...,0,0,0,0,0,0,57,88,307,57
4,/Prime_FULL/02-010/W0/OD/4.tif,5,0,0,0,0,1,0.0,0.0,0.0,...,0,1,0,0,0,0,57,88,307,57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3915,/Prime_FULL/02-046/W104/OD/44.tif,45,0,0,0,0,0,0.0,0.0,0.0,...,0,0,0,0,0,0,96,82,259,96
3916,/Prime_FULL/02-046/W104/OD/45.tif,46,0,0,0,0,0,0.0,0.0,0.0,...,0,0,0,0,0,0,96,82,259,96
3917,/Prime_FULL/02-046/W104/OD/46.tif,47,0,0,0,0,0,0.0,0.0,0.0,...,0,0,0,0,0,0,96,82,259,96
3918,/Prime_FULL/02-046/W104/OD/47.tif,48,0,0,0,0,0,0.0,0.0,0.0,...,0,0,0,0,0,0,96,82,259,96


In [29]:
# 提取各部分
biomarker_df['Patient_ID_String'] = biomarker_df['Path (Trial/Arm/Folder/Visit/Eye/Image Name)'].str.extract(r'/(\d{2}-\d{3})/')
biomarker_df['Week'] = biomarker_df['Path (Trial/Arm/Folder/Visit/Eye/Image Name)'].str.extract(r'/W(\d+)/')[0].astype(float)
biomarker_df['Eye'] = biomarker_df['Path (Trial/Arm/Folder/Visit/Eye/Image Name)'].str.extract(r'/(OD|OS)/')[0]
biomarker_df['OCT_Filename'] = biomarker_df['Path (Trial/Arm/Folder/Visit/Eye/Image Name)'].str.split('/').str[-1]

# 提取B-scan编号（从文件名）
biomarker_df['B_scan_number'] = pd.to_numeric(
    biomarker_df['OCT_Filename'].str.extract(r'^(\d+)\.')[0],
    errors='coerce'
)
biomarker_df.to_csv('biomarker_processed.csv', index=False)
biomarker_df



,Path (Trial/Arm/Folder/Visit/Eye/Image Name),Scan (n/49),Atrophy / thinning of retinal layers,Disruption of EZ,DRIL,IR hemorrhages,IR HRF,Partially attached vitreous face,Fully attached vitreous face,Preretinal tissue/hemorrhage,...,SHRM,Eye_ID,BCVA,CST,Patient_ID,Patient_ID_String,Week,Eye,OCT_Filename,B_scan_number
0,/Prime_FULL/02-010/W0/OD/0.tif,1,0,0,0,0,1,0.0,0.0,0.0,...,0,57,88,307,57,02-010,0.0,OD,0.tif,0
1,/Prime_FULL/02-010/W0/OD/1.tif,2,0,0,0,0,1,0.0,0.0,0.0,...,0,57,88,307,57,02-010,0.0,OD,1.tif,1
2,/Prime_FULL/02-010/W0/OD/2.tif,3,0,0,0,0,1,0.0,0.0,0.0,...,0,57,88,307,57,02-010,0.0,OD,2.tif,2
3,/Prime_FULL/02-010/W0/OD/3.tif,4,0,0,0,0,1,0.0,0.0,0.0,...,0,57,88,307,57,02-010,0.0,OD,3.tif,3
4,/Prime_FULL/02-010/W0/OD/4.tif,5,0,0,0,0,1,0.0,0.0,0.0,...,0,57,88,307,57,02-010,0.0,OD,4.tif,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3915,/Prime_FULL/02-046/W104/OD/44.tif,45,0,0,0,0,0,0.0,0.0,0.0,...,0,96,82,259,96,02-046,104.0,OD,44.tif,44
3916,/Prime_FULL/02-046/W104/OD/45.tif,46,0,0,0,0,0,0.0,0.0,0.0,...,0,96,82,259,96,02-046,104.0,OD,45.tif,45
3917,/Prime_FULL/02-046/W104/OD/46.tif,47,0,0,0,0,0,0.0,0.0,0.0,...,0,96,82,259,96,02-046,104.0,OD,46.tif,46
3918,/Prime_FULL/02-046/W104/OD/47.tif,48,0,0,0,0,0,0.0,0.0,0.0,...,0,96,82,259,96,02-046,104.0,OD,47.tif,47
